In [2]:
import os
import pandas as pd
from models import prep

pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)

df = prep.load_dataset(prep.import_directory, prep.lotti_fn, prep.vincitori_fn)
# df = prep.split_sum_totals(df)
# df = prep.feature_extraction(df)
# df = prep.remove_obvious_outliers(df)

/Users/nepal/Documents/synapthesis/models/prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[temp.index, 3] = temp.values


In [7]:
df = df[~df.duplicated()]
temp = df[df["id_lotto"].duplicated(keep=False)]
temp = temp.join(temp.groupby("id_lotto").size().rename("n_winners"), on="id_lotto")
temp = temp["importo"] / temp["n_winners"]

In [14]:
df.loc[temp.index, "importo"] = temp

In [ ]:
def split_sum_totals(df):
    # remove duplicates
    df = df[~df.duplicated()]
    # save records having more than one winner in a temporary variable
    temp = df[df["id_lotto"].duplicated(keep=False)]
    # equally split the lot sum among all the winners
    # (uniform distr. is uninformative prior)
    temp = temp.join(temp.groupby("id_lotto").size().rename("n_winners"),
                     on="id_lotto")
    temp = temp["importo"] / temp["n_winners"]
    # modifico la copia o l'originale?
    # l'originale secondo quello che so
    df.loc[temp.index, "importo"] = temp
    return df